In [2]:
import json
import os
from pathlib import Path
from py2neo import Graph

with open("config.json") as f:
    config = json.load(f)

temp_dir = "temp"
os.makedirs(temp_dir, exist_ok=True)

port = config["port"]
user = config["user"]
pswd = config["pswd"]
neo4j_import_loc = Path(config["neo4j_import_loc"])

graph = Graph("bolt://localhost:" + port, auth=(user, pswd))

In [16]:
graph.run(
    """
        MATCH (co2:Compound {inChI: "InChI=1S/CO2/c2-1-3"})
        MATCH (ethanol:Compound {inChI: "InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3"})
        MATCH (co2)-[:INPUT {role: REACTANT}]->(r1)-[:OUTPUT]->(n:Compound)-[:INPUT {role: REACTANT}]->(r2)-[:OUTPUT]->(ethanol)
        CALL {
            WITH r1
            MATCH (r1)<-[:INPUT]-(input1:Compound)
            RETURN collect(input1.inChI) AS inputs1
        }
        CALL {
            WITH r1
            MATCH (r1)-[:OUTPUT]->(output1:Compound)
            RETURN collect(output1.inChI) AS outputs1
        }
        CALL {
            WITH r2
            MATCH (r2)<-[:INPUT]-(input1:Compound)
            RETURN collect(input1.inChI) AS inputs2
        }
        CALL {
            WITH r2
            MATCH (r2)-[:OUTPUT]->(output1:Compound)
            RETURN collect(output1.inChI) AS outputs2
        }
        RETURN inputs1, r1.reactionId, outputs1, inputs2, r2.reactionId, outputs2 limit 1
    """
).data()

[{'inputs1': ['InChI=1S/C4H2F6O2/c5-3(6,7)1-12-2(11)4(8,9)10/h1H2',
   'InChI=1S/C5H5N.ClH/c1-2-4-6-5-3-1;/h1-5H;1H/p+1',
   'InChI=1S/CO2/c2-1-3',
   'InChI=1S/C2H3F3O/c3-2(4,5)1-6/h6H,1H2',
   'InChI=1S/C4F6O3/c5-3(6,7)1(11)13-2(12)4(8,9)10'],
  'r1.reactionId': 'ord-571db4c759b54fbcba683499835041c0',
  'outputs1': ['InChI=1S/C5H5N.C2HF3O2/c1-2-4-6-5-3-1;3-2(4,5)1(6)7/h1-5H;(H,6,7)',
   'InChI=1S/ClH/h1H'],
  'inputs2': ['InChI=1S/C25H21ClN4O3/c1-14-10-11-19(12-15(14)2)30-25(33)22(16(3)29-30)28-27-21-9-5-8-20(23(21)31)17-6-4-7-18(13-17)24(26)32/h4-13,27,31H,1-3H3/b28-22-',
   'InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3',
   'InChI=1S/C5H9NO.H3N/c1-6-4-2-3-5(6)7;/h2-4H2,1H3;1H3',
   'InChI=1S/ClH/h1H'],
  'r2.reactionId': 'ord-956856757f7142f283df6730c7a6a802',
  'outputs2': ['InChI=1S/C25H23N5O3/c1-14-10-11-19(12-15(14)2)30-25(33)22(16(3)29-30)28-27-21-9-5-8-20(23(21)31)17-6-4-7-18(13-17)24(26)32/h4-13,27,31H,1-3H3,(H2,26,32)',
   'InChI=1S/C2H6O/c1-2-3/h3H,2H2,1H3']}]